In [1]:
import seaborn as sns
from tqdm import tqdm
import pandas as pd
from rpy2.robjects import r, pandas2ri
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

In [2]:
pandas2ri.activate()

In [3]:
from isaac.constants import BASIC_TRAINING_COLS

In [4]:
def are_all_coordinates_within_bounds(trial):
    return ((trial[["o1.x", "o2.x", "o3.x", "o4.x", "o1.y", "o2.y", "o3.y", "o4.y"]] < 0) |
           (trial[["o1.x", "o2.x", "o3.x", "o4.x"]] > 6) |
           (trial[["o1.y", "o2.y", "o3.y", "o4.y"]] > 4)).any().any()

# Experiment 1 

In [5]:
rdata_path = "data/experiment1.rdata"
r['load'](rdata_path)

'df.l','df.w','ls.clips','ls.events'


In [6]:
all_trials = r['ls.clips']

responses = r['df.l'].query("yoker == 'NA'")[["trueMass", "trueRelationship"]]
YOKED_PATH = "data/yoked_trials_exp1.h5"
ALTERNATIVE_TRAINING_COLS = ["object"+str(obj)+"."+attr for obj in range(1, 5) for attr in ["x", "y", "vx", "vy"]]

/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


In [7]:
if os.path.exists(YOKED_PATH):
    os.remove(YOKED_PATH)

In [11]:
mass_trials = []
force_trials = []

trial_counter = 0

for participant in tqdm(all_trials):
    for trial in participant:
        trial = pandas2ri.ri2py_dataframe(trial)
            
        if set(ALTERNATIVE_TRAINING_COLS).issubset(trial.columns):
            trial = trial.rename({alt_col: col for col, alt_col in 
                                  zip(BASIC_TRAINING_COLS, ALTERNATIVE_TRAINING_COLS)}, axis="columns")
        else:
            print("Not all columns are available for participant")
            continue
            
        if not are_all_coordinates_within_bounds:
            print("Skipping trial because coordinates out of bounds")
            continue
     
        trial = trial[BASIC_TRAINING_COLS].copy()

        true_mass = responses.iloc[trial_counter]["trueMass"]
        trial["A"] = (true_mass == "A")
        trial["B"] = (true_mass == "B")
        trial["same"] = (true_mass == "same")

        true_relationship = responses.iloc[trial_counter]["trueRelationship"]
        trial["attract"] = (true_relationship == "attract")
        trial["none"] = (true_relationship == "none")
        trial["repel"] = (true_relationship == "repel")

        trial.to_hdf(path_or_buf=YOKED_PATH, key="trial_"+str(trial_counter))

        trial_counter += 1

  0%|          | 0/44 [00:00<?, ?it/s]/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)
 75%|███████▌  | 33/44 [00:07<00:02,  5.18it/s]

Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for 

 80%|███████▉  | 35/44 [00:07<00:01,  6.37it/s]

Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for 

 89%|████████▊ | 39/44 [00:08<00:00,  8.71it/s]

Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for 

 98%|█████████▊| 43/44 [00:08<00:00, 10.53it/s]

Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for 

100%|██████████| 44/44 [00:08<00:00,  5.21it/s]

Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant


# Experiment 2 

In [12]:
MASS_PATH = "data/mass_trials_exp2.h5"
FORCE_PATH = "data/force_trials_exp2.h5"

rdata_path = "data/experiment2.rdata"
r['load'](rdata_path)

'df.l','df.w','ls.clips','ls.events'


In [13]:
responses = r['df.l'].query("experiment == 'experiment_4'")[["trueMass", "trueRelationship"]]
all_trials = r['ls.clips']

/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


In [14]:
if os.path.exists(MASS_PATH):
    os.remove(MASS_PATH)

if os.path.exists(FORCE_PATH):
    os.remove(FORCE_PATH)

In [15]:
mass_trial_counter = -1
force_trial_counter = -1

for participant in tqdm(all_trials):
    for trial in participant:
        trial = pandas2ri.ri2py_dataframe(trial)
        
        if trial.question.unique()[0] == "mass":
            hdf_path = MASS_PATH
            mass_trial_counter += 1
            trial_counter = mass_trial_counter
        else:
            hdf_path = FORCE_PATH
            force_trial_counter += 1
            trial_counter = force_trial_counter
        
        if not are_all_coordinates_within_bounds:
            if trial.question.unique()[0] == "mass":
                print("Skipping trial for mass")
                mass_trial_counter -= 1
            elif trial.question.unique()[0] == "force":
                print("Skipping trial for force")
                force_trial_counter -= 1
            
            continue

        trial = trial[BASIC_TRAINING_COLS].copy()
        
        
        true_mass = responses.iloc[trial_counter]["trueMass"]
        trial["A"] = (true_mass == "A")
        trial["B"] = (true_mass == "B")
        trial["same"] = (true_mass == "same")

        true_relationship = responses.iloc[trial_counter]["trueRelationship"]
        trial["attract"] = (true_relationship == "attract")
        trial["none"] = (true_relationship == "none")
        trial["repel"] = (true_relationship == "repel")

        trial.to_hdf(path_or_buf=hdf_path, key="trial_"+str(trial_counter))


  0%|          | 0/40 [00:00<?, ?it/s]/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)
100%|██████████| 40/40 [00:16<00:00,  2.09it/s]


# READING DATASET

In [17]:
from isaac.dataset import read_dataset

In [18]:
yoked_dataset = read_dataset(MASS_PATH)

In [19]:
yoked_dataset[0].head()

,o1.x,o1.y,o1.vx,o1.vy,o2.x,o2.y,o2.vx,o2.vy,o3.x,o3.y,...,o4.x,o4.y,o4.vx,o4.vy,A,B,same,attract,none,repel
0,4.39098,2.94941,-6.10030,7.70026,1.11518,0.67968,-8.28633,-1.52380,2.49884,2.18351,...,2.80825,2.19310,6.74918,-9.15409,True,False,False,False,True,False
1,4.28943,3.07765,-6.09304,7.69472,0.97720,0.65431,-8.27925,-1.52207,2.46567,2.02650,...,2.93350,2.04108,6.74497,-9.14519,True,False,False,False,True,False
2,4.18800,3.20581,-6.08604,7.68941,0.83933,0.62898,-8.27198,-1.52016,2.44526,1.86996,...,3.04585,1.88880,6.74074,-9.13658,True,False,False,False,True,False
3,4.08668,3.33388,-6.07936,7.68424,0.70159,0.60367,-8.26453,-1.51819,2.42478,1.71348,...,3.15812,1.73667,6.73645,-9.12822,True,False,False,False,True,False
4,3.98546,3.46187,-6.07299,7.67911,0.56397,0.57840,-8.25689,-1.51622,2.40424,1.55706,...,3.27032,1.58467,6.73205,-9.12005,True,False,False,False,True,False


In [20]:
df_list = read_dataset("data/passive_trials.h5")

In [21]:
df_list[0].head()

,o1.x,o1.y,o1.vx,o1.vy,o2.x,o2.y,o2.vx,o2.vy,o3.x,o3.y,...,o4.x,o4.y,o4.vx,o4.vy,A,B,same,attract,none,repel
0,4.607864,3.095769,-3.778900,5.391553,0.648429,2.192734,3.909157,7.427639,1.522031,2.620404,...,0.778790,0.474302,0.068809,-1.523547,False,True,False,True,False,False
1,4.544752,3.185518,-3.786723,5.384933,0.714367,2.316823,3.956280,7.445341,1.561135,2.743228,...,0.779989,0.449083,0.071921,-1.513125,False,True,False,True,False,False
2,4.481496,3.275155,-3.795316,5.378207,0.781130,2.441237,4.005784,7.464850,1.598596,2.864995,...,0.779989,0.473645,0.000000,1.473736,False,True,False,True,False,False
3,4.418084,3.364677,-3.804760,5.371369,0.848766,2.566012,4.058147,7.486512,1.634338,2.985648,...,0.780031,0.498310,0.002533,1.479906,False,True,False,True,False,False
4,4.354498,3.454084,-3.815150,5.364408,0.917332,2.691192,4.113963,7.510764,1.668263,3.105112,...,0.780112,0.523069,0.004877,1.485504,False,True,False,True,False,False
